# Part 3. DEAP Dataset + Asymmetry + SVM

In this part 3, we will focus on feature engineering using asymmetry analysis.  Asymmetry analysis here refers to the analysis of imbalance between left and right symmetrical location.

Asymmetry analysis is another very basic and must-do analysis for emotions/cognitions/resting state.

In this part, we shall extract these asymmetries as features and then input these features into SVM and see if these features are useful for predicting the four valence-arousal classes that we have obtained from Part 1.

In [1]:
import torch

import os
import pickle
import numpy as np

Set cuda accordingly.

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Configured device: ", device)

Configured device:  cuda


## 1. Loading dataset

Let's first reuse the dataset loader we have created in Part 1.  *Note that we gonna focus on only the valence dataset for this tutorial.*

In [3]:
class Dataset(torch.utils.data.Dataset):
    
    def __init__(self, path, stim):
        _, _, filenames = next(os.walk(path))
        filenames = sorted(filenames)
        all_data = []
        all_label = []
        for dat in filenames:
            temp = pickle.load(open(os.path.join(path,dat), 'rb'), encoding='latin1')
            all_data.append(temp['data'])
            
            if stim == "Valence":
                all_label.append(temp['labels'][:,:1])   #the first index is valence
            elif stim == "Arousal":
                all_label.append(temp['labels'][:,1:2]) # Arousal  #the second index is arousal
                
        self.data = np.vstack(all_data)[:, :32, ]   #shape: (1280, 32, 8064) --> take only the first 32 channels
        self.label = np.vstack(all_label) #(1280, )  ==> 1280 samples, each with a unique label (depend on the param "stim")
        
        del temp, all_data, all_label

    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, idx):
        single_data  = self.data[idx]
        single_label = (self.label[idx] > 5).astype(float)   #convert the scale to either 0 or 1 (to classification problem)
        
        batch = {
            'data': torch.Tensor(single_data),
            'label': torch.Tensor(single_label)
        }
        
        return batch

Let's try load the dataset.

In [4]:
path = "data"  #create a folder "data", and inside put s01.dat,....,s32.dat inside from the preprocessed folder from the DEAP dataset

In [5]:
dataset = Dataset(path, "Valence")

data  = dataset[:]['data']
label = dataset[:]['label']

print("Data shape: " , data.shape)  #1280 = 32 * 40 trials, 32 EEG channels, 8064 samples
print("Label shape: ", label.shape)  #four classes of LALV, HALV, LAHV, HAHV

Data shape:  torch.Size([1280, 32, 8064])
Label shape:  torch.Size([1280, 1])


Let's look the label distribution of the dataset.

In [6]:
lv = label == 0
hv = label == 1

assert len(label[lv]) + len(label[hv]) == label.shape[0]  #simple unit test
print("count of low valence: ", len(label[lv]))
print("count of high valence: ", len(label[hv]))

count of low valence:  572
count of high valence:  708


Let's see the median of EEG of each group (you can do std on your own exercise)

In [7]:
lv_unsqueeze = lv.squeeze()
hv_unsqueeze = hv.squeeze()

print("Median of low valence",  np.median(data[lv_unsqueeze, :, :]))
print("Median of high valence", np.median(data[hv_unsqueeze, :, :]))

Median of low valence 0.009302851
Median of high valence 0.0034587365


## 2. Asymmetry Analysis

## 3. Machine Learning